In [22]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [23]:
log = pd.DataFrame(pd.read_csv("/kaggle/input/fantasy-premier-league-dataset-2023-2024/players.csv"))
log.sample(5)

,id,name,now_cost,position,team,chance_of_playing_this_round,starts,total_points,news_added,form,...,form_rank,creativity_rank,threat,points_per_game,saves_per_90,ict_index,clean_sheets_per_90,influence_rank,assists,dreamteam_count
420,329,Fred Onyedinma,50,MID,Luton,0.0,0,0,2023-07-31T07:05:08.472622Z,0.0,...,694,714,0.0,0.0,0.0,0.0,0.00,716,0,0
477,370,Alex Telles,45,DEF,Man Utd,0.0,0,0,2023-07-24T09:56:30.018038Z,0.0,...,576,612,0.0,0.0,0.0,0.0,0.00,616,0,0
532,418,Garang Kuol,45,MID,Newcastle,0.0,0,0,2023-08-09T08:37:02.624952Z,0.0,...,644,673,0.0,0.0,0.0,0.0,0.00,676,0,0
528,414,Jeff Hendrick,44,MID,Newcastle,0.0,0,0,2023-09-02T08:49:55.025666Z,0.0,...,526,564,0.0,0.0,0.0,0.0,0.00,575,0,0
560,378,Anthony Elanga,50,MID,Nott'm Forest,100.0,6,39,2023-11-16T15:00:09.725767Z,5.0,...,31,104,207.0,3.2,0.0,51.7,0.14,112,3,0


# **Team in 23/24**

In [24]:
team = log["team"].values
All_team = np.unique(team)
All_team

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Liverpool', 'Luton', 'Man City', 'Man Utd', 'Newcastle',
       "Nott'm Forest", 'Sheffield Utd', 'Spurs', 'West Ham', 'Wolves'],
      dtype=object)

# **Team Stadium**

In [25]:
text = open("/kaggle/input/team-stadium/Team_Stadium.txt").read()
# Split the string into lines
# lines = teams_and_stadiums.strip().split('\n')

# # Create the mapping dictionary
# # lines.remove('')
# mapping_dict = dict(zip(lines[0::2], lines[1::2]))

# # Print the mapping dictionary
# print(mapping_dict)

lines = [line.strip() for line in text.strip().split('\n') if line.strip()]
team_stadium_list = [(lines[i], lines[i + 1]) for i in range(0, len(lines), 2)]
team_stadium = np.array(team_stadium_list)

team_stadium[:5]

array([['Arsenal', 'Emirates Stadium'],
       ['Aston Villa', 'Villa Park'],
       ['Barnsley', 'Oakwell'],
       ['Birmingham City', "St. Andrew's Trillion Trophy Stadium"],
       ['Blackburn Rovers', 'Ewood Park']], dtype='<U36')

# **Get only played Team Stadium**

In [26]:
# Drop unplayed team
df = pd.DataFrame(team_stadium)
# df = np.delete(df,3)
df = df.drop([2,3,4,5,6,8,12,13,15,17,20,21,22,23,24,29,31,33,34,35,36,38,39,40,41,42,43,45,46,48,49])
df

,0,1
0,Arsenal,Emirates Stadium
1,Aston Villa,Villa Park
7,Bournemouth,Vitality Stadium
9,Brentford,Gtech Community Stadium
10,Brighton & Hove Albion,Amex Stadium
11,Burnley,Turf Moor
14,Chelsea,Stamford Bridge
16,Crystal Palace,Selhurst Park
18,Everton,Goodison Park
19,Fulham,Craven Cottage


In [27]:
# Turn in to array
df = np.array(df)
print(df[:5])

[['Arsenal' 'Emirates Stadium']
 ['Aston Villa' 'Villa Park']
 ['Bournemouth' 'Vitality Stadium']
 ['Brentford' 'Gtech Community Stadium']
 ['Brighton & Hove Albion' 'Amex Stadium']]


# **Scraping player data**

# Get all player

In [39]:
import requests
from bs4 import BeautifulSoup

# Function to extract team ID from the given URL
def extract_team_id(url):
    # Extract team ID from the URL
    team_id = url.split('/')[2]
    return team_id

# URL of the page containing club information
url = "https://www.premierleague.com/clubs"

# Fetch the content of the page
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract team IDs from each club card
club_cards = soup.find_all('li', class_='club-card-wrapper')
team_ids = [extract_team_id(club_card.find('a')['href']) for club_card in club_cards]

# Display the list of team IDs
team_id = np.column_stack((df, team_ids))
team_id = np.array(team_id)
team_id

array([['Arsenal', 'Emirates Stadium', '1'],
       ['Aston Villa', 'Villa Park', '2'],
       ['Bournemouth', 'Vitality Stadium', '127'],
       ['Brentford', 'Gtech Community Stadium', '130'],
       ['Brighton & Hove Albion', 'Amex Stadium', '131'],
       ['Burnley', 'Turf Moor', '43'],
       ['Chelsea', 'Stamford Bridge', '4'],
       ['Crystal Palace', 'Selhurst Park', '6'],
       ['Everton', 'Goodison Park', '7'],
       ['Fulham', 'Craven Cottage', '34'],
       ['Liverpool', 'Anfield', '10'],
       ['Luton Town', 'Kenilworth Road', '163'],
       ['Manchester City', 'Etihad Stadium', '11'],
       ['Manchester United', 'Old Trafford', '12'],
       ['Newcastle United', "St. James' Park", '23'],
       ['Nottingham Forest', 'The City Ground', '15'],
       ['Sheffield United', 'Bramall Lane', '18'],
       ['Tottenham Hotspur', 'Tottenham Hotspur Stadium', '21'],
       ['West Ham United', 'London Stadium', '25'],
       ['Wolverhampton Wanderers', 'Molineux Stadium', '38']]

In [ ]:
team_id[]

In [40]:
import bs4
import requests
from datetime import datetime

for i in range(team_id.shape[0]):
    url = f"https://www.premierleague.com/clubs/{team_id[i, 2]}/squad?se=578"
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    pl = []

    # Find all player information cards
    player_cards = soup.find_all('li', class_='stats-card')

    # Function to extract date of birth from player's overview page and convert it to the desired format
    def get_date_of_birth(player_url):
        player_page = requests.get(f"https://www.premierleague.com{player_url}")
        player_soup = bs4.BeautifulSoup(player_page.text, 'html.parser')

        # Find the div with label "Date of Birth" and extract the date
        dob_element = player_soup.find('div', {'class': 'player-overview__label'}, string='Date of Birth')
        if dob_element:
            dob_text = dob_element.find_next('div', {'class': 'player-overview__info'}).get_text(strip=True)
            # Parse the date string and format it
            dob_date = datetime.strptime(dob_text.split('(')[0].strip(), '%d/%m/%Y').strftime('%Y-%m-%d')
        else:
            dob_date = ''

        return dob_date

    # Iterate through each player card and extract information
    for card in player_cards:
        name_first_element = card.select_one('.stats-card__player-first')
        name_last_element = card.select_one('.stats-card__player-last')
        position_element = card.select_one('.stats-card__player-position')
        squad_number_element = card.select_one('.stats-card__squad-number')
        player_url = card.select_one('.stats-card__wrapper')['href']

        # Check if elements exist before extracting text
        name_first = name_first_element.get_text(strip=True) if name_first_element else ''
        name_last = name_last_element.get_text(strip=True) if name_last_element else ''
        position = position_element.get_text(strip=True) if position_element else ''
        squad_number = squad_number_element.get_text(strip=True) if squad_number_element else ''

        # Extract date of birth using the function
        dob = get_date_of_birth(player_url)

        pl.append([name_first, name_last, position, squad_number, dob])

    # Print the list of player information
    pl[:5]
    
    # Sql Insertaion

    # Team ID
    team_id = i+1

    # Generate a single SQL INSERT statement for all players
    sql_query = (
        "INSERT INTO `player` "
        "(`firstName`, `middleName`, `lastName`, "
        "`shirtNo`, `birthDate`, `position`, `isBanned`, "
        "`isInjured`, `teamID`) "
        "VALUES "
    )

    # Add values for each player
    for player_info in pl:
        sql_query += "('{}', NULL, '{}', '{}', '{}', '{}', NULL, NULL, '{}'), ".format(
            player_info[0], player_info[1], player_info[3],player_info[4], player_info[2], team_id
        )

    # Remove the trailing comma and print the final SQL query
    sql_query = sql_query.rstrip(', ')
    print(sql_query + ';')

INSERT INTO `player` (`firstName`, `middleName`, `lastName`, `shirtNo`, `birthDate`, `position`, `isBanned`, `isInjured`, `teamID`) VALUES;


TypeError: 'int' object is not subscriptable